In [1]:
import pickle 
import pandas as pd

with open("titanic_step4_importance_train.pickle", "rb") as pickle_file:
    train_importance = pickle.load(pickle_file)

with open("titanic_step4_importance_train_y.pickle", "rb") as pickle_file:
    train_y = pickle.load(pickle_file)

with open("titanic_step4_importance_test.pickle", "rb") as pickle_file:
    test_importance = pickle.load(pickle_file)



In [2]:
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings

warnings.filterwarnings("ignore")


#### SVC 하이퍼 파라미터
- SVC는 분류 경계가 최대가 되는 하이퍼 플레인을 구하는 것이 목표
- 하이퍼 플레인을 일직선 또는 곡률을 가진 선으로 할지 선택 가능
- 주요 하이퍼 파라미터
    - C: regularization 
    - gamma: 얼마나 훈련 데이터셋에 fit하게 할 지 결정

#### RandomizedSearchCV + SVC
- Grid Search보다 보통 성능이 높다고 평가됨
- 하지만 수행 시간이 상당히 걸릴 수 있음 

In [5]:
hyper_params = {
    "C": stats.uniform(0, 50),
    "gamma": stats.uniform(0, 1)
}

gd = RandomizedSearchCV(
    estimator=SVC(random_state=2025),
    param_distributions=hyper_params,
    n_iter=100,
    cv=5, # 내부적으로 Stratified K-Fold 사용
    scoring="accuracy",
    random_state=2025,
    n_jobs=-1
)

gd.fit(train_importance, train_y)
print(gd.best_score_)
print(gd.best_params_)

df = pd.DataFrame(gd.cv_results_)
print(df[["params", "mean_test_score"]])

0.8339087314041805
{'C': np.float64(4.889400554283291), 'gamma': np.float64(0.1516847042112811)}
                                               params  mean_test_score
0   {'C': 6.77440818389809, 'gamma': 0.88785170273...         0.817086
1   {'C': 46.630281994325124, 'gamma': 0.445568164...         0.821549
2   {'C': 19.41177730569913, 'gamma': 0.2575964353...         0.822666
3   {'C': 32.86837927355188, 'gamma': 0.4926169375...         0.821549
4   {'C': 48.21192096250286, 'gamma': 0.8009844748...         0.820444
..                                                ...              ...
95  {'C': 14.63028322882729, 'gamma': 0.7955577851...         0.820444
96  {'C': 17.165184253749988, 'gamma': 0.156595114...         0.819315
97  {'C': 31.259768801363897, 'gamma': 0.212003570...         0.822666
98  {'C': 27.108447939999504, 'gamma': 0.875683423...         0.819321
99  {'C': 28.86465178921625, 'gamma': 0.4122925309...         0.821549

[100 rows x 2 columns]


#### GridSearchCV + SVC
- 하이퍼 파라미터는 일반적으로 적절한 범위가 존재하지 않음, 성능이 나오는 범위를 감에 의존
- Randomized Search를 통해 대략적인 범위를 알아낸 후, Grid Search로 범위와 최적의 값을 지정

In [5]:
hyper_params = {
    "C": [10, 15, 20, 23, 25, 30, 50],
    "gamma": [0.001, 0.01, 0.05, 0.06, 0.07, 1]
}

gd = GridSearchCV(
    estimator=SVC(random_state=2025),
    param_grid=hyper_params,
    verbose=True,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)

gd.fit(train_importance, train_y)
print(gd.best_score_)
print(gd.best_params_)


Fitting 5 folds for each of 42 candidates, totalling 210 fits
0.8294080723118448
{'C': 15, 'gamma': 0.07}


#### Graident Boosting Classifier 하이퍼 파파미터
- learning_rate: 학습률, 가중치를 업데이트할 때 마다 어느정도로 수정할 지의 비율
- n_estimator: 트리의 개수 
- max_depth: 트리의 깊이

#### Gradient Boosting Classifier + GridSearch

In [7]:
hyper_params = {
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "n_estimators" : [100, 1000, 2000],
    "max_depth":  [3, 5, 10, 15]
}

gd = GridSearchCV(
    estimator=GradientBoostingClassifier(random_state=2025),
    param_grid=hyper_params,
    verbose=5,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
)

gd.fit(train_importance, train_y)
print(gd.best_score_)
print(gd.best_params_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
0.8316740945326722
{'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 2000}


#### Logisitic Regression 하이퍼 파라미터 
- penalty: regularization 종류 
- C: regularization 적용 강도

In [8]:
hyper_params = {
    "penalty": ["l1", "l2", "elasticnet"],
    "C": stats.uniform(0, 1000)
}

gd = RandomizedSearchCV(
    estimator=LogisticRegression(random_state=2025),
    param_distributions=hyper_params,
    cv=5,
    n_iter=100,
    scoring="accuracy",
    random_state=2025,
    n_jobs=-1
)

gd.fit(train_importance, train_y)
print(gd.best_score_)
print(gd.best_params_)

0.8317054798819911
{'C': np.float64(126.408546896565), 'penalty': 'l2'}


In [9]:
hyper_params = {
    "penalty": ["l1", "l2"],
    "C": np.linspace(700, 900, 200)
}

gd = GridSearchCV(
    estimator=LogisticRegression(random_state=2025),
    param_grid=hyper_params,
    verbose=5,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
)

gd.fit(train_importance, train_y)
print(gd.best_score_)
print(gd.best_params_)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits
0.8317054798819911
{'C': np.float64(701.0050251256281), 'penalty': 'l2'}
